In [ ]:
# Add the relevant scripts from LArMachineLearningData
# Nice the process so it can run with lots of cores on low priority
import os
os.nice(20)

# Add local paths
import sys
hnlDIR = os.environ['_']
sys.path.append('../pyscript')

# From pyscript Library
from Plotting import *
from Dictionary import *
from HelperFunctions import *
from CutFunctions import *

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

<h1>Configuration Stuff Here</h1>

In [ ]:
ifSave = True
savePath = "../plot_files/28Feb2024_m200_swap_cut_order/"

In [ ]:
ifCut = True

In [ ]:
ifTune = False

TuneCut = {
    'crumbs': 0 #0.14
    , 'Muon' : 0.04
    , 'Proton' : 9.8 
    , 'Pion' : 0.8
    #, 'Opt0Score': 1550
    , 'Opt0FracMore': -0.2
    , 'Opt0FracLess': 0.68
    , 'OneShowerTheta': 18
    , 'TwoShowerTheta': 24
    }
    

<h1> Read in PKL Dataframe </h1>

<h3> HNL </h3>

In [ ]:
df_hnl = pd.DataFrame()

for i in range(0, 5):
    hnl_path = "../pkl_files/v2/hnl_m200_50k_v2_slc_{}.pkl".format(i)
    print(hnl_path)
    
    file = open(hnl_path, 'rb')
    df = pickle.load(file)
    
    df_hnl = pd.concat((df_hnl, df), ignore_index=True)
    file.close()
    
#--------------------------------------------------#
dfsubrun_hnl = pd.DataFrame()

for i in range(0, 5):
    hnl_path = "../pkl_files/v2/hnl_m200_50k_v2_subrun_{}.pkl".format(i)
    print(hnl_path)
    
    file = open(hnl_path, 'rb')
    df = pickle.load(file)
    
    dfsubrun_hnl = pd.concat((dfsubrun_hnl, df), ignore_index=True)
    file.close()

<h3> TPC Neutrino </h3>

In [ ]:
df_nu = pd.DataFrame()

for i in range(0, 10):
    nu_path = "../pkl_files/v2/nu_rockbox_100k_v2_slc_{}.pkl".format(i)
    print(nu_path)
    
    file = open(nu_path, 'rb')
    df = pickle.load(file)
    
    df_nu = pd.concat((df_nu, df), ignore_index=True)
    file.close()
    
#--------------------------------------------------#
dfsubrun_nu = pd.DataFrame()

for i in range(0, 10):
    nu_path = "../pkl_files/v2/nu_rockbox_100k_v2_subrun_{}.pkl".format(i)
    print(nu_path)
    
    file = open(nu_path, 'rb')
    df = pickle.load(file)
    
    dfsubrun_nu = pd.concat((dfsubrun_nu, df), ignore_index=True)
    file.close()

<h3> Intime Cosmics </h3>

In [ ]:
df_cos = pd.DataFrame()

for i in range(0, 5):
    cos_path = "../pkl_files/v2/intime_cosmics_50k_v2_slc_{}.pkl".format(i)
    print(cos_path)
    
    file = open(cos_path, 'rb')
    df = pickle.load(file)
    
    df_cos = pd.concat((df_cos, df), ignore_index=True)
    file.close()
    
#--------------------------------------------------#
dfsubrun_cos = pd.DataFrame()

for i in range(0, 5):
    cos_path = "../pkl_files/v2/intime_cosmics_50k_v2_subrun_{}.pkl".format(i)
    print(cos_path)
    
    file = open(cos_path, 'rb')
    df = pickle.load(file)
    
    dfsubrun_cos = pd.concat((dfsubrun_cos, df), ignore_index=True)
    file.close()

<h1>Some Scaling Voodoo</h1>

<h3>HNL</h3>

In [ ]:
true_signals = 7005 + 6986 + 6923 + 7010 + 6909
true_nonfv_signals = 3045 + 3014 + 3077 + 2990 + 2991
total_true_signals = 10050 + 10000 + 10000 + 10000 + 9900
start_signals = 6668 + 6680 + 6626 + 6701 + 6591
start_nonfv_signals = 2568 + 2530 + 2569 + 2515 + 2502
total_start_signals = 9236 + 9210 + 9195 + 9216 + 9093

In [ ]:
#Sanity Check: ifScale == 1 i.e. Umu = 1e-7 and 3 years POT
#true counts ~1.5k
#start counts ~1.4k
scale_pot_hnl, hnl_spill = calc_scaling_pot(dfsubrun_hnl, df_hnl, ifScale = 1)

true_signals = true_signals * scale_pot_hnl
true_nonfv_signals = true_nonfv_signals * scale_pot_hnl
total_true_signals = total_true_signals * scale_pot_hnl
start_signals = start_signals * scale_pot_hnl
start_nonfv_signals = start_nonfv_signals * scale_pot_hnl
total_start_signals = total_start_signals * scale_pot_hnl

true_counts = total_true_signals
start_counts = total_start_signals
print("true counts = " + str(true_counts))
print("start counts = " + str(start_counts))

In [ ]:
simU = 1e-7 #already squared
scaleHNLPlot = 80

In [ ]:
plotU = getUfromScaleFactor(simU, scaleHNLPlot)
true_counts = true_counts * scaleHNLPlot
start_counts =  start_counts * scaleHNLPlot

print("Scale Umu by " + str(scaleHNLPlot))
print("Scaled Umu = " + str(plotU))
print("Scaled true counts = " + str(true_counts))
print("Scaled start counts = " + str(start_counts))

In [ ]:
scale_pot_hnl, hnl_spill = calc_scaling_pot(dfsubrun_hnl, df_hnl, ifScale = scaleHNLPlot)

del dfsubrun_hnl

<h3>Rock Box Neutrino</h3>

In [ ]:
scale_pot_nu, nu_spill = calc_scaling_pot(dfsubrun_nu, df_nu)

del  dfsubrun_nu

<h3>Intime Cosmics</h3>

In [ ]:
scale_pot_cosmics = calc_scaling_spill(dfsubrun_cos, df_cos, hnl_spill, nu_spill)

del  dfsubrun_cos

<h1> Make Beam Bucket - Pre Cosmics </h1>

In [ ]:
width = 19.00

df_hnl['mod_t'] = df_hnl['slc_opt0_time_corrected_Z_pandora']%width
df_nu['mod_t'] = df_nu['slc_opt0_time_corrected_Z_pandora']%width
df_cos['mod_t'] = df_cos['slc_opt0_time_corrected_Z_pandora']%width
#----------------------------------------------
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_precosmics.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> Save Purity and Eff </h1>

In [ ]:
purity_arr, eff_arr, cut_arr = [], [], []

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)


purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Clear Cosmics Removal")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> CRUMBS </h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_crumbs_score', 
                    scaleHNLPlot,
                    xmin = -2.5, xmax =1, xnbin = 40,
                    xtitle = 'CRUMBS Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"crumbs_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
if ifTune:
    vary_crumbsScore(df_hnl, df_nu, df_cos, true_counts, start_counts)
    
    if ifSave:
        plt.savefig(savePath+"/crumbs_tuning.png", dpi =200)

<h3>Apply Cut</h3>

In [ ]:
#score = TuneCut['crumbs']
score = 0.00
print(score)

if ifCut:
    df_hnl = cutCosmics(df_hnl, crumbsScore = score)
    df_nu = cutCosmics(df_nu, crumbsScore = score)
    df_cos = cutCosmics(df_cos, crumbsScore = score)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("CRUMBS")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_crumbs_score', 
                    scaleHNLPlot,
                    xmin = -2.5, xmax =1, xnbin = 40,
                    xtitle = 'CRUMBS Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"crumbs_postcut.png", dpi = 200)
#-----------------------------------#

plt.show()

<h1> Make Beam Bucket - Post Crumbs</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]"
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postcosmics_postcrumbs.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> Fiducial Volume </h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_is_fv', 
                    scaleHNLPlot,
                    xmin = -1, xmax = 3, xnbin = 40,
                    xtitle = 'Vertex is in Fiducial Volume'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"fv_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Apply Cut</h3>

In [ ]:
if ifCut:
    df_hnl = cutFV(df_hnl)
    df_nu = cutFV(df_nu)
    df_cos = cutFV(df_cos)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Vertex In FV")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_is_fv', 
                    scaleHNLPlot,
                    xmin = -1, xmax = 3, xnbin = 40,
                    xtitle = 'Vertex is in Fiducial Volume'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"fv_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> Make Beam Bucket - Post FV</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]"
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postcosmics_postFV.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Muon Rejection</h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_muons', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 5,
                    xtitle = 'n Razzled Muons'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_muon_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_muon_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Muon Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_muon_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
if ifTune:
    vary_MuonScore(df_hnl, df_nu, df_cos , true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/muon_tuning.png", dpi =200)

<h3> Apply Cut</h3>

In [ ]:
#muonScore = TuneCut['Muon']
muonScore = 0.04
print(muonScore)

if ifCut:
    df_hnl = cutMuon(df_hnl, muonScore =muonScore)
    df_nu = cutMuon(df_nu, muonScore =muonScore)
    df_cos = cutMuon(df_cos,muonScore =muonScore)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Muon Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_muons', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 5,
                    xtitle = 'n Razzled Muons'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_muon_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_muon_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Muon Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_muon_score_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> Make Beam Bucket - Post Muon</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]"
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postcosmics_postmuon.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Proton Rejection</h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_protons_thresh', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 5,
                    xtitle = 'n Razzled Protons'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_proton_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_proton_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Proton Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_proton_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
if ifTune:
    vary_ProtonScore(df_hnl, df_nu, df_cos, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/proton_tuning.png", dpi =200)

<h3>Apply Cut</h3>

In [ ]:
#protonScore = TuneCut['Proton']
protonScore = 1
print(protonScore)

if ifCut:
    df_hnl = cutProton(df_hnl, nProton = 0, protonScore = protonScore)
    df_nu = cutProton(df_nu, nProton = 0, protonScore = protonScore)
    df_cos = cutProton(df_cos, nProton = 0, protonScore = protonScore)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Proton Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_protons_thresh', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 5,
                    xtitle = 'n Razzled Protons'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_proton_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_proton_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Proton Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_proton_score_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> Make Beam Bucket - Post Proton</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]"
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postcosmics_postproton.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Pion Rejection</h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_pions_thresh', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 5,
                    xtitle = 'n Razzled Pions'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_pion_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_pion_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Pion Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_pion_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
if ifTune:
    vary_PionScore(df_hnl, df_nu, df_cos, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/pion_tuning.png", dpi =200)

<h3>Apply Cut </h3>

In [ ]:
#pionScore = TuneCut['Pion']
pionScore = 0.8
print(pionScore)

if ifCut:
    df_hnl = cutPion(df_hnl, nPion = 0, pionScore = pionScore)
    df_nu = cutPion(df_nu, nPion = 0, pionScore = pionScore)
    df_cos = cutPion(df_cos, nPion = 0, pionScore = pionScore)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Pion Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_pions_thresh', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 5,
                    xtitle = 'n Razzled Pions'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_pion_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_pion_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Pion Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_pion_score_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> Make Beam Bucket - Post Pion</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + " [ns]"
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postcosmics_postpion.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Electron Rejection (?)</h1>

<h3> All PFP Shower </h3>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_electrons', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 5,
                    xtitle = 'n Razzled Electrons'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_electron_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_electron_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Electron Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_electron_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3> Get Primary Shower </h3>

In [ ]:
df_hnl_prim = get_primary_shw_df(df_hnl)

df_nu_prim = get_primary_shw_df(df_nu)

df_cos_prim = get_primary_shw_df(df_cos)

<h3> Primary Shower </h3>

In [ ]:
plot_slc_var(df_hnl_prim, df_nu_prim, df_cos_prim,
                    true_counts, start_counts, 
                    'slc_n_razzled_electrons', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 5,
                    xtitle = 'n Razzled Electrons'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_electron_prim_shw_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl_prim[df_hnl_prim['slc_pfp_razzled_electron_score'] < 0.95]
            , df_nu_prim[df_nu_prim['slc_pfp_razzled_electron_score'] < 0.95]
             , df_cos_prim[df_cos_prim['slc_pfp_razzled_electron_score'] < 0.95],
                    true_counts, start_counts, 
                    'slc_pfp_razzled_electron_score', 
                    scaleHNLPlot,
                    xmin = 0.90, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Electron Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_electron_score_prim_shw_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Photon Selection (?)</h1>

<h3> All PFP Shower </h3>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_n_razzled_photons', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 10, xnbin = 10,
                    xtitle = 'n Razzled Photons'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_photon_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_photon_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Photon Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_photon_score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3> Primary Shower </h3>

In [ ]:
plot_slc_var(df_hnl_prim, df_nu_prim, df_cos_prim,
                    true_counts, start_counts, 
                    'slc_n_razzled_photons', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 10, xnbin = 10,
                    xtitle = 'n Razzled Photons'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"nrazzled_photon_prim_shw_precut.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl_prim, df_nu_prim, df_cos_prim,
                    true_counts, start_counts, 
                    'slc_pfp_razzled_photon_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 1, xnbin = 40,
                    xtitle = 'PFP Razzled Photon Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"razzled_photon_score_prim_shw_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Multiplicity</h1>

<h1> Opt0 Score </h1>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_opt0_score', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 40000, xnbin = 40,
                    xtitle = 'Opt0 Score'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0score_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3> Apply Cut </h3>

<h3>Save Purity/Eff</h3>

<h1>Opt0 Frac </h1>

<h3>Both 1 and 2+ Shower in Slice</h3>

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_opt0_frac', 
                    scaleHNLPlot,
                    xmin = -1, xmax = 2, xnbin = 40,
                    xtitle = 'Opt0 Fraction'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0frac_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
if ifTune:
    vary_OpT0FracMore(df_hnl, df_nu, df_cos, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/opt0frac_more_tuning.png", dpi =200)

In [ ]:
if ifTune:
    vary_OpT0FracLess(df_hnl, df_nu, df_cos, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/opt0frac_less_tuning.png", dpi =200)

<h3> Split 1 and 2+ shower cases </h3>

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

<h3> 1 shower cases </h3>

In [ ]:
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
                    true_counts, start_counts, 
                    'slc_opt0_frac', 
                    scaleHNLPlot,
                    xmin = -1, xmax = 2, xnbin = 40,
                    xtitle = 'Opt0 Fraction'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0frac_one_shw_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
if ifTune:
    vary_OpT0FracMore(df_hnl_shw1, df_nu_shw1, df_cos_shw1, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/opt0frac_more_one_shw_tuning.png", dpi =200)

In [ ]:
if ifTune:
    vary_OpT0FracLess(df_hnl_shw1, df_nu_shw1, df_cos_shw1, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/opt0frac_less_one_shw_tuning.png", dpi =200)

<h3> 2+ shower cases </h3>

In [ ]:
plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos_shw2,
                    true_counts, start_counts, 
                    'slc_opt0_frac', 
                    scaleHNLPlot,
                    xmin = -1, xmax = 2, xnbin = 40,
                    xtitle = 'Opt0 Fraction'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0frac_two_shw_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
if ifTune:
    vary_OpT0FracMore(df_hnl_shw2, df_nu_shw2, df_cos_shw2, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/opt0frac_more_two_shw_tuning.png", dpi =200)

In [ ]:
if ifTune:
    vary_OpT0FracLess(df_hnl_shw2, df_nu_shw2, df_cos_shw2, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/opt0frac_less_two_shw_tuning.png", dpi =200)

<h3> Apply Cut </h3>

<h3> 1 shower only: harsh cut</h3>

In [ ]:
#fracMore = TuneCut['Opt0FracMore']
fracMore = -0.14
print(fracMore)
fracLess = 0.6
print(fracLess)

if ifCut:
    df_hnl_shw1 = cutOpt0Frac(df_hnl_shw1, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_nu_shw1 = cutOpt0Frac(df_nu_shw1, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_cos_shw1 = cutOpt0Frac(df_cos_shw1, Opt0FracMore = fracMore, Opt0FracLess = fracLess)

<h3> 2+ showers: cut on primary + less harsh cut </h3>

In [ ]:
#fracMore = TuneCut['Opt0FracMore']
fracMore = 0
print(fracMore)
fracLess = 0.6
print(fracLess)

if ifCut:
    df_hnl_shw2 = cutOpt0Frac(df_hnl_shw2, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_nu_shw2 = cutOpt0Frac(df_nu_shw2, Opt0FracMore = fracMore, Opt0FracLess = fracLess)
    df_cos_shw2 = cutOpt0Frac(df_cos_shw2, Opt0FracMore = fracMore, Opt0FracLess = fracLess)

In [ ]:
df_hnl = apply_shower_cut(df_hnl, df_hnl_shw1, df_hnl_shw2)

df_nu = apply_shower_cut(df_nu, df_nu_shw1, df_nu_shw2)

df_cos = apply_shower_cut(df_cos, df_cos_shw1, df_cos_shw2)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Opt0 Fraction")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'slc_opt0_frac', 
                    scaleHNLPlot,
                    xmin = -1, xmax = 2, xnbin = 40,
                    xtitle = 'Opt0 Fraction'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"opt0frac_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1> Make Beam Bucket - Post Opt0 Frac</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]'
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postopt0.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Shower Theta</h1>

<h3> Get Primary Shower </h3>

In [ ]:
df_hnl_prim = get_primary_shw_df(df_hnl)

df_nu_prim = get_primary_shw_df(df_nu)

df_cos_prim = get_primary_shw_df(df_cos)

<h3> Primary Showers of both 1 and 2+ shower cases </h3>

In [ ]:
plot_slc_var(df_hnl_prim, df_nu_prim, df_cos_prim,
#plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_theta', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 180, xnbin = 20,
                    xtitle = 'Shower Theta'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"shower_theta_precut.png", dpi = 200)
#-----------------------------------#
plt.show()


In [ ]:
if ifTune:
    vary_Theta(df_hnl_prim, df_nu_prim, df_cos_prim, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/prim_shower_theta_tuning.png", dpi =200)

<h3> Split 1 and 2+ shower cases </h3>

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

<h3> Primary Showers of 1 shower cases </h3>

In [ ]:
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
#plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_theta', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 180, xnbin = 20,
                    xtitle = 'Shower Theta'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"one_shower_theta_precut.png", dpi = 200)
#-----------------------------------#
plt.show()


In [ ]:
if ifTune:
    vary_Theta(df_hnl_shw1, df_nu_shw1, df_cos_shw1, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/one_shower_theta_tuning.png", dpi =200)

<h3> All Showers of 2+ shower cases </h3>

In [ ]:
df_nu_shw2 = df_nu_shw2.apply(pd.Series.explode).reset_index()
df_hnl_shw2 = df_hnl_shw2.apply(pd.Series.explode).reset_index()
df_cos_shw2 = df_cos_shw2.apply(pd.Series.explode).reset_index()

In [ ]:
plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos_shw2,
#plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_theta', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 180, xnbin = 20,
                    xtitle = 'Shower Theta'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_shower_theta_precut.png", dpi = 200)
#-----------------------------------#
plt.show()


In [ ]:
if ifTune:
    vary_Theta(df_hnl_shw2, df_nu_shw2, df_cos_shw2, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/two_shower_theta_tuning.png", dpi =200)

<h3> Primary Showers of 2+ shower cases </h3>

In [ ]:
df_hnl_prim_shw2 = get_primary_shw_df(df_hnl_shw2)

df_nu_prim_shw2 = get_primary_shw_df(df_nu_shw2)

df_cos_prim_shw2 = get_primary_shw_df(df_cos_shw2)

In [ ]:
plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
#plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_theta', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 180, xnbin = 20,
                    xtitle = 'Shower Theta'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_shower_primary_theta_precut.png", dpi = 200)
#-----------------------------------#
plt.show()


In [ ]:
if ifTune:
    vary_Theta(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/two_shower_prim_theta_tuning.png", dpi =200)

<h3>Apply Cut</h3>

<h3> 1 shower only: harsh cut</h3>

In [ ]:
#angle = TuneCut['OneShowerTheta']
angle = 19
print(angle)

if ifCut:
    df_hnl_shw1 = cutThetaAngle(df_hnl_shw1, thetaAngle = angle) 
    df_nu_shw1 = cutThetaAngle(df_nu_shw1, thetaAngle = angle) 
    df_cos_shw1 = cutThetaAngle(df_cos_shw1, thetaAngle = angle) 
#df_cos = cutThetaAngle(df_cos, thetaAngle = angle) 

<h3> 2+ showers: cut on primary + less harsh cut </h3>

In [ ]:
#angle = TuneCut['TwoShowerTheta']
angle = 25
print(angle)

if ifCut:
    df_hnl_prim_shw2 = cutThetaAngle(df_hnl_prim_shw2, thetaAngle = angle) 
    df_nu_prim_shw2 = cutThetaAngle(df_nu_prim_shw2, thetaAngle = angle) 
    df_cos_prim_shw2 = cutThetaAngle(df_cos_prim_shw2, thetaAngle = angle) 
#df_cos = cutThetaAngle(df_cos, thetaAngle = angle) 

In [ ]:
df_hnl_shw2 = merge_df_prim(df_hnl_shw2, df_hnl_prim_shw2)

df_nu_shw2 = merge_df_prim(df_nu_shw2, df_nu_prim_shw2)

df_cos_shw2 = merge_df_prim(df_cos_shw2, df_cos_prim_shw2)

In [ ]:
df_hnl = apply_shower_cut(df_hnl, df_hnl_shw1, df_hnl_shw2)

df_nu = apply_shower_cut(df_nu, df_nu_shw1, df_nu_shw2)

df_cos = apply_shower_cut(df_cos, df_cos_shw1, df_cos_shw2)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Shower Theta Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Shower Theta</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]'
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postshowertheta.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Shower Energy and Conversion Gap</h1>

<h3> Get Primary Shower </h3>

In [ ]:
df_hnl_prim = get_primary_shw_df(df_hnl)

df_nu_prim = get_primary_shw_df(df_nu)

df_cos_prim = get_primary_shw_df(df_cos)

<h3> Primary Showers of both 1 and 2+ shower cases </h3>

In [ ]:
plot_slc_var(df_hnl_prim, df_nu_prim, df_cos_prim,
                    true_counts, start_counts, 
                    'slc_pfp_shower_energy', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 2000, xnbin = 100,
                    xtitle = 'Shower Energy [MeV]'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_shower_energy_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()
#-----------------------------------#
plot_slc_var(df_hnl_prim, df_nu_prim, df_cos_prim,
                    true_counts, start_counts, 
                    'slc_pfp_shower_conv_gap', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 40,
                    xtitle = 'Shower Conversion Gap [cm]'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"prim_shower_convgap_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
if ifTune:
    vary_ShowerEnergy(df_hnl_prim, df_nu_prim, df_cos_prim, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/prim_shower_energy_tuning.png", dpi =200)

<h3> Split into 1 shower and multi shower case </h3>

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

#df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

<h3> Primary Showers of 1 shower cases </h3>

In [ ]:
#plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_energy', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 2000, xnbin = 100,
                    xtitle = 'Shower Energy [MeV]'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"one_shower_energy_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos_shw1,
#plot_slc_var(df_hnl_shw1, df_nu_shw1, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_conv_gap', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 40,
                    xtitle = 'Shower Conversion Gap [cm]'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"one_shower_convgap_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
if ifTune:
    vary_ShowerEnergy(df_hnl_shw1, df_nu_shw1, df_cos_shw1, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/prim_one_shower_energy_tuning.png", dpi =200)

<h3> All Showers of 2+ shower cases </h3>

In [ ]:
df_nu_shw2 = df_nu_shw2.apply(pd.Series.explode).reset_index()
df_hnl_shw2 = df_hnl_shw2.apply(pd.Series.explode).reset_index()
#df_cos_shw2 = df_cos_shw2.apply(pd.Series.explode).reset_index()

<h3> All Showers of 2+ shower cases </h3>

In [ ]:
#plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos_shw2,
plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_energy', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 2000, xnbin = 100,
                    xtitle = 'Shower Energy [MeV]'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_shower_energy_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
#plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos_shw2,
plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos,
                    true_counts, start_counts, 
                    'slc_pfp_shower_conv_gap', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 40,
                    xtitle = 'Shower Conversion Gap [cm]'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_shower_convgap_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
if ifTune:
    #vary_ShowerEnergy(df_hnl_shw2, df_nu_shw2, df_cos_shw2, true_counts, start_counts)
    vary_ShowerEnergy(df_hnl_shw2, df_nu_shw2, df_cos, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/two_shower_energy_tuning.png", dpi =200)

<h3> Primary Showers of 2+ shower cases </h3>

In [ ]:
df_hnl_prim_shw2 = get_primary_shw_df(df_hnl_shw2)

df_nu_prim_shw2 = get_primary_shw_df(df_nu_shw2)

#df_cos_prim_shw2 = get_primary_shw_df(df_cos_shw2)
df_cos_prim_shw2 = get_primary_shw_df(df_cos)

In [ ]:
#plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos_shw2,
plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
                    true_counts, start_counts, 
                    'slc_pfp_shower_energy', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 2000, xnbin = 100,
                    xtitle = 'Shower Energy [MeV]'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_prim_shower_energy_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()
#----------------------------------------------
#plot_slc_var(df_hnl_shw2, df_nu_shw2, df_cos_shw2,
plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
                    true_counts, start_counts, 
                    'slc_pfp_shower_conv_gap', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 5, xnbin = 40,
                    xtitle = 'Shower Conversion Gap [cm]'
                    )
#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"two_prim_shower_convgap_postpid.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
if ifTune:
    vary_ShowerEnergy(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/prim_two_shower_energy_tuning.png", dpi =200)

<h3>Apply Cut</h3>

<h3> 1 shower only: harsh cut</h3>

<h3> 2+ showers: cut on primary + less harsh cut </h3>

<h3>Save Purity/Eff</h3>

<h1> Make Beam Bucket - Post Shower Energy</h1>

<h1>Combine Opt0 + Shower Theta Cut</h1>

<h3> Primary Showers of both 1 and 2+ shower cases </h3>

In [ ]:
df_hnl_prim = get_primary_shw_df(df_hnl)

df_nu_prim = get_primary_shw_df(df_nu)

df_cos_prim = get_primary_shw_df(df_cos)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4), sharey= True, sharex = True)

xmin, xmax = -1,2
xnbin = 90

ymin, ymax = 0, 180
ynbin = 90

xlimmin, xlimmax = xmin, xmax
ylimmin, ylimmax = ymin, ymax
#----------------------------------------------
plot_2dhist( df_nu_prim['slc_opt0_frac'], df_nu_prim['slc_pfp_shower_theta'], 
            ax1, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            iflog = True, cmin= 1, cmax =200,
            #weights = df_nu['scale_pot'],
            title = 'Neutrino', xtitle = 'Opt0 Fraction', ytitle= 'Shower Theta'
                )
#----------------------------------------------
plot_2dhist( df_hnl_prim['slc_opt0_frac'], df_hnl_prim['slc_pfp_shower_theta'], 
            ax2, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            #weights = df_hnl['scale_pot'],
            iflog = True, cmin= 1, cmax =200,
            title = 'HNL', xtitle = 'Opt0 Fraction', ytitle= ''
                )
#----------------------------------------------
plot_2dhist( df_cos_prim['slc_opt0_frac'], df_cos_prim['slc_pfp_shower_theta'], 
            ax3, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            #weights = df_cos['scale_pot'],
            iflog = True, cmin= 1, cmax =200,
            title = 'Intime Cosmics', xtitle = 'Opt0 Fraction', ytitle= ''
                )
#----------------------------------------------
#plt.ylim(0,40)

ax1.tick_params(bottom = True, top = False, left = True, right = False)
ax1.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'out')

ax2.tick_params(bottom = True, top = False, left = True, right = False)
ax2.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax2.tick_params(axis = 'y', labelsize = 16, direction = 'out')

ax3.tick_params(bottom = True, top = False, left = True, right = False)
ax3.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax3.tick_params(axis = 'y', labelsize = 16, direction = 'out')

#----------------------------------------------
#plt.savefig(savePath+"opt0Score_shower_like.png", dpi=200)
plt.show()

In [ ]:
ifTune = True

if ifTune:
    vary_ThetaFrac(df_hnl_prim, df_nu_prim, df_cos_prim, true_counts, start_counts)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4), sharey= True, sharex = True)

xmin, xmax = -1,2
xnbin = 90

ymin, ymax = 0, 180
ynbin = 90

xlimmin, xlimmax = xmin, xmax
ylimmin, ylimmax = ymin, ymax
#----------------------------------------------
plot_2dhist( df_nu_prim['slc_opt0_frac'], df_nu_prim['slc_pfp_shower_theta'], 
            ax1, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            iflog = True, cmin= 1, cmax =200,
            #weights = df_nu['scale_pot'],
            title = 'Neutrino', xtitle = 'Opt0 Fraction', ytitle= 'Shower Theta'
                )
#----------------------------------------------
plot_2dhist( df_hnl_prim['slc_opt0_frac'], df_hnl_prim['slc_pfp_shower_theta'], 
            ax2, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            #weights = df_hnl['scale_pot'],
            iflog = True, cmin= 1, cmax =200,
            title = 'HNL', xtitle = 'Opt0 Fraction', ytitle= ''
                )
#----------------------------------------------
plot_2dhist( df_cos_prim['slc_opt0_frac'], df_cos_prim['slc_pfp_shower_theta'], 
            ax3, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            #weights = df_cos['scale_pot'],
            iflog = True, cmin= 1, cmax =200,
            title = 'Intime Cosmics', xtitle = 'Opt0 Fraction', ytitle= ''
                )
#----------------------------------------------
#plt.ylim(0,40)

ax1.tick_params(bottom = True, top = False, left = True, right = False)
ax1.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'out')

ax2.tick_params(bottom = True, top = False, left = True, right = False)
ax2.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax2.tick_params(axis = 'y', labelsize = 16, direction = 'out')

ax3.tick_params(bottom = True, top = False, left = True, right = False)
ax3.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax3.tick_params(axis = 'y', labelsize = 16, direction = 'out')
#----------------------------------------------
# Creating vectors X and Y
x = np.linspace(-1, 2, 100)
y = ( x + 1 ) * 20 - 4
 
# Create the plot
ax2.plot(x, y, color='r')
#----------------------------------------------
#plt.savefig(savePath+"opt0Score_shower_like.png", dpi=200)
plt.show()

<h3> Split into 1 shower and multi shower case </h3>

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

<h3> Primary Showers of 1 shower cases </h3>

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4), sharey= True, sharex = True)

xmin, xmax = -1,2
xnbin = 90

ymin, ymax = 0, 180
ynbin = 90

xlimmin, xlimmax = xmin, xmax
ylimmin, ylimmax = ymin, ymax
#----------------------------------------------
plot_2dhist( df_nu_shw1['slc_opt0_frac'], df_nu_shw1['slc_pfp_shower_theta'], 
            ax1, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            iflog = True, cmin= 1, cmax =200,
            #weights = df_nu['scale_pot'],
            title = 'Neutrino', xtitle = 'Opt0 Fraction', ytitle= 'Shower Theta'
                )
#----------------------------------------------
plot_2dhist( df_hnl_shw1['slc_opt0_frac'], df_hnl_shw1['slc_pfp_shower_theta'], 
            ax2, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            #weights = df_hnl['scale_pot'],
            iflog = True, cmin= 1, cmax =200,
            title = 'HNL', xtitle = 'Opt0 Fraction', ytitle= ''
                )
#----------------------------------------------
plot_2dhist( df_cos_shw1['slc_opt0_frac'], df_cos_shw1['slc_pfp_shower_theta'], 
            ax3, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            #weights = df_cos['scale_pot'],
            iflog = True, cmin= 1, cmax =200,
            title = 'Intime Cosmics', xtitle = 'Opt0 Fraction', ytitle= ''
                )
#----------------------------------------------
#plt.ylim(0,40)

ax1.tick_params(bottom = True, top = False, left = True, right = False)
ax1.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'out')

ax2.tick_params(bottom = True, top = False, left = True, right = False)
ax2.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax2.tick_params(axis = 'y', labelsize = 16, direction = 'out')

ax3.tick_params(bottom = True, top = False, left = True, right = False)
ax3.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax3.tick_params(axis = 'y', labelsize = 16, direction = 'out')

#----------------------------------------------
#plt.savefig(savePath+"opt0Score_shower_like.png", dpi=200)
plt.show()

In [ ]:
ifTune = True

if ifTune:
    vary_ThetaFrac(df_hnl_shw1, df_nu_shw1, df_cos_shw1, true_counts, start_counts)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4), sharey= True, sharex = True)

xmin, xmax = -1,2
xnbin = 90

ymin, ymax = 0, 180
ynbin = 90

xlimmin, xlimmax = xmin, xmax
ylimmin, ylimmax = ymin, ymax
#----------------------------------------------
plot_2dhist( df_nu_shw1['slc_opt0_frac'], df_nu_shw1['slc_pfp_shower_theta'], 
            ax1, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            iflog = True, cmin= 1, cmax =200,
            #weights = df_nu['scale_pot'],
            title = 'Neutrino', xtitle = 'Opt0 Fraction', ytitle= 'Shower Theta'
                )
#----------------------------------------------
plot_2dhist( df_hnl_shw1['slc_opt0_frac'], df_hnl_shw1['slc_pfp_shower_theta'], 
            ax2, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            #weights = df_hnl['scale_pot'],
            iflog = True, cmin= 1, cmax =200,
            title = 'HNL', xtitle = 'Opt0 Fraction', ytitle= ''
                )
#----------------------------------------------
plot_2dhist( df_cos_shw1['slc_opt0_frac'], df_cos_shw1['slc_pfp_shower_theta'], 
            ax3, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            #weights = df_cos['scale_pot'],
            iflog = True, cmin= 1, cmax =200,
            title = 'Intime Cosmics', xtitle = 'Opt0 Fraction', ytitle= ''
                )
#----------------------------------------------
#plt.ylim(0,40)

ax1.tick_params(bottom = True, top = False, left = True, right = False)
ax1.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'out')

ax2.tick_params(bottom = True, top = False, left = True, right = False)
ax2.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax2.tick_params(axis = 'y', labelsize = 16, direction = 'out')

ax3.tick_params(bottom = True, top = False, left = True, right = False)
ax3.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax3.tick_params(axis = 'y', labelsize = 16, direction = 'out')
#----------------------------------------------
# Creating vectors X and Y
x = np.linspace(-1, 2, 100)
y = ( x + 1 ) * 15 - 2
 
# Create the plot
ax2.plot(x, y, color='r')
#----------------------------------------------
#plt.savefig(savePath+"opt0Score_shower_like.png", dpi=200)
plt.show()

<h3> Primary Showers of 2+ shower cases </h3>

In [ ]:
df_nu_shw2 = df_nu_shw2.apply(pd.Series.explode).reset_index()
df_hnl_shw2 = df_hnl_shw2.apply(pd.Series.explode).reset_index()
df_cos_shw2 = df_cos_shw2.apply(pd.Series.explode).reset_index()

In [ ]:
df_hnl_prim_shw2 = get_primary_shw_df(df_hnl_shw2)

df_nu_prim_shw2 = get_primary_shw_df(df_nu_shw2)

df_cos_prim_shw2 = get_primary_shw_df(df_cos_shw2)
#df_cos_prim_shw2 = get_primary_shw_df(df_cos)

In [ ]:
df_hnl_prim_shw2

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4), sharey= True, sharex = True)

xmin, xmax = -1,2
xnbin = 90

ymin, ymax = 0, 180
ynbin = 90

xlimmin, xlimmax = xmin, xmax
ylimmin, ylimmax = ymin, ymax
#----------------------------------------------
plot_2dhist( df_nu_prim_shw2['slc_opt0_frac'], df_nu_prim_shw2['slc_pfp_shower_theta'], 
            ax1, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            iflog = True, cmin= 1, cmax =200,
            #weights = df_nu['scale_pot'],
            title = 'Neutrino', xtitle = 'Opt0 Fraction', ytitle= 'Shower Theta'
                )
#----------------------------------------------
plot_2dhist( df_hnl_prim_shw2['slc_opt0_frac'], df_hnl_prim_shw2['slc_pfp_shower_theta'], 
            ax2, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            #weights = df_hnl['scale_pot'],
            iflog = True, cmin= 1, cmax =200,
            title = 'HNL', xtitle = 'Opt0 Fraction', ytitle= ''
                )
#----------------------------------------------
plot_2dhist( df_cos_prim_shw2['slc_opt0_frac'], df_cos_prim_shw2['slc_pfp_shower_theta'], 
            ax3, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            #weights = df_cos['scale_pot'],
            iflog = True, cmin= 1, cmax =200,
            title = 'Intime Cosmics', xtitle = 'Opt0 Fraction', ytitle= ''
                )
#----------------------------------------------
#plt.ylim(0,40)

ax1.tick_params(bottom = True, top = False, left = True, right = False)
ax1.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'out')

ax2.tick_params(bottom = True, top = False, left = True, right = False)
ax2.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax2.tick_params(axis = 'y', labelsize = 16, direction = 'out')

ax3.tick_params(bottom = True, top = False, left = True, right = False)
ax3.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax3.tick_params(axis = 'y', labelsize = 16, direction = 'out')

#----------------------------------------------
#plt.savefig(savePath+"opt0Score_shower_like.png", dpi=200)
plt.show()

In [ ]:
ifTune = True

if ifTune:
    vary_ThetaFrac(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2, true_counts, start_counts)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4), sharey= True, sharex = True)

xmin, xmax = -1,2
xnbin = 90

ymin, ymax = 0, 180
ynbin = 90

xlimmin, xlimmax = xmin, xmax
ylimmin, ylimmax = ymin, ymax
#----------------------------------------------
plot_2dhist( df_nu_prim_shw2['slc_opt0_frac'], df_nu_prim_shw2['slc_pfp_shower_theta'], 
            ax1, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            iflog = True, cmin= 1, cmax =200,
            #weights = df_nu['scale_pot'],
            title = 'Neutrino', xtitle = 'Opt0 Fraction', ytitle= 'Shower Theta'
                )
#----------------------------------------------
plot_2dhist( df_hnl_prim_shw2['slc_opt0_frac'], df_hnl_prim_shw2['slc_pfp_shower_theta'], 
            ax2, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            #weights = df_hnl['scale_pot'],
            iflog = True, cmin= 1, cmax =200,
            title = 'HNL', xtitle = 'Opt0 Fraction', ytitle= ''
                )
#----------------------------------------------
plot_2dhist( df_cos_prim_shw2['slc_opt0_frac'], df_cos_prim_shw2['slc_pfp_shower_theta'], 
            ax3, fig, 
            xmin, xmax, ymin, ymax, xnbin, ynbin,
            xlimmin, xlimmax, ylimmin, ylimmax,
            ifcbar =True,
            #weights = df_cos['scale_pot'],
            iflog = True, cmin= 1, cmax =200,
            title = 'Intime Cosmics', xtitle = 'Opt0 Fraction', ytitle= ''
                )
#----------------------------------------------
#plt.ylim(0,40)

ax1.tick_params(bottom = True, top = False, left = True, right = False)
ax1.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'out')

ax2.tick_params(bottom = True, top = False, left = True, right = False)
ax2.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax2.tick_params(axis = 'y', labelsize = 16, direction = 'out')

ax3.tick_params(bottom = True, top = False, left = True, right = False)
ax3.tick_params(axis = 'x', labelsize = 16, direction = 'out')
ax3.tick_params(axis = 'y', labelsize = 16, direction = 'out')
#----------------------------------------------
# Creating vectors X and Y
x = np.linspace(-1, 2, 100)
y = ( x + 1 ) * 19 - 0
 
# Create the plot
ax2.plot(x, y, color='r')
#----------------------------------------------
#plt.savefig(savePath+"opt0Score_shower_like.png", dpi=200)
plt.show()

<h3>Apply Cut</h3>

<h3> 1 shower only: harsh cut</h3>

In [ ]:
#harsh: m = 10, c = 0
m=15
c=-2

if ifCut:
    df_hnl_shw1 = cutThetaFrac(df_hnl_shw1, m, c)
    df_nu_shw1 = cutThetaFrac(df_nu_shw1, m, c)
    df_cos_shw1 = cutThetaFrac(df_cos_shw1, m, c)

<h3> 2+ showers: cut on primary + less harsh cut </h3>

In [ ]:
#angle = TuneCut['TwoShowerTheta']
m=19
c=0

if ifCut:
    df_hnl_prim_shw2 = cutThetaFrac(df_hnl_prim_shw2, m, c)
    df_nu_prim_shw2 = cutThetaFrac(df_nu_prim_shw2, m, c)
    df_cos_prim_shw2 = cutThetaFrac(df_cos_prim_shw2, m, c)

In [ ]:
df_hnl_shw2 = merge_df_prim(df_hnl_shw2, df_hnl_prim_shw2)

df_nu_shw2 = merge_df_prim(df_nu_shw2, df_nu_prim_shw2)

df_cos_shw2 = merge_df_prim(df_cos_shw2, df_cos_prim_shw2)

In [ ]:
df_hnl = apply_shower_cut(df_hnl, df_hnl_shw1, df_hnl_shw2)

df_nu = apply_shower_cut(df_nu, df_nu_shw1, df_nu_shw2)

df_cos = apply_shower_cut(df_cos, df_cos_shw1, df_cos_shw2)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Combined Theta + Opt0 Frac Cut")

print(purity_arr)
print(eff_arr)
print(cut_arr)

<h1> Make Beam Bucket - Post Shower Energy</h1>

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]'
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_postCombinedThetaFrac.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>2 Shower Case: Pi0 Mass </h1>

<h3> Split into 1 shower and multi shower case again! </h3>

In [ ]:
df_hnl_shw1, df_hnl_shw2 = split_my_df(df_hnl)

df_nu_shw1, df_nu_shw2 = split_my_df(df_nu)

#df_cos_shw1, df_cos_shw2 = split_my_df(df_cos)

In [ ]:
df_hnl_shw2['pi0_mass'] = df_hnl_shw2.apply(lambda row: pi0mass(row), axis = 1)

df_nu_shw2['pi0_mass'] = df_nu_shw2.apply(lambda row: pi0mass(row), axis = 1)

df_cos['pi0_mass']  = ""
#df_cos_shw2['pi0_mass'] = df_cos_shw2.apply(lambda row: pi0mass(row), axis = 1)


In [ ]:
df_hnl_prim_shw2 = get_primary_shw_df(df_hnl_shw2)

df_nu_prim_shw2 = get_primary_shw_df(df_nu_shw2)

#df_cos_prim_shw2 = get_primary_shw_df(df_cos_shw2)
df_cos_prim_shw2 = get_primary_shw_df(df_cos)

In [ ]:
plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos,
#plot_slc_var(df_hnl_prim_shw2, df_nu_prim_shw2, df_cos_prim_shw2,
                    true_counts, start_counts, 
                    'pi0_mass', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 600, xnbin = 50,
                    xtitle = 'Pi 0 Mass [MeV]'
            )
# only one line may be specified; full height
plt.axvline(x = 135, color = col_dict['Aqua'], linewidth = 3)

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"pizero_mass_precut.png", dpi = 200)
#-----------------------------------#

plt.show()

<h3>Apply Cut</h3>

<h3>Merge The Cut Back</h3>

<h3>Save Purity/Eff</h3>

<h1> Make Beam Bucket - Post PiZeroMass</h1>

<h1>Electron</h1>

In [ ]:
df_hnl_prim = get_primary_shw_df(df_hnl)

df_nu_prim = get_primary_shw_df(df_nu)

df_cos_prim = get_primary_shw_df(df_cos)

In [ ]:
width = 19.00

df_hnl_prim['mod_t'] = df_hnl_prim['slc_opt0_time_corrected_Z_pandora']%width
df_nu_prim['mod_t'] = df_nu_prim['slc_opt0_time_corrected_Z_pandora']%width
df_cos_prim['mod_t'] = df_cos_prim['slc_opt0_time_corrected_Z_pandora']%width
#----------------------------------------------

hist, bins = plot_slc_var(
                    df_hnl_prim
                    , df_nu_prim
                    , df_cos_prim
                    , true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]'
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_razzled_electron_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h3>Apply Cut</h3>

In [ ]:
if ifCut:
    df_hnl_prim = df_hnl_prim[df_hnl_prim['slc_pfp_razzled_electron_score'] < 0.98]
    df_nu_prim = df_nu_prim[df_nu_prim['slc_pfp_razzled_electron_score'] < 0.98]
    df_cos_prim = df_cos_prim[df_cos_prim['slc_pfp_razzled_electron_score'] < 0.98]

In [ ]:
df_hnl = merge_df_prim(df_hnl_shw2, df_hnl_prim_shw2)

df_nu_shw2 = merge_df_prim(df_nu_shw2, df_nu_prim_shw2)

df_cos_shw2 = merge_df_prim(df_cos_shw2, df_cos_prim_shw2)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Between Bucket")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
width = 19.00

df_hnl_prim['mod_t'] = df_hnl_prim['slc_opt0_time_corrected_Z_pandora']%width
df_nu_prim['mod_t'] = df_nu_prim['slc_opt0_time_corrected_Z_pandora']%width
df_cos_prim['mod_t'] = df_cos_prim['slc_opt0_time_corrected_Z_pandora']%width
#----------------------------------------------

hist, bins = plot_slc_var(
                    df_hnl_prim
                    , df_nu_prim
                    , df_cos_prim
                    , true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]'
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_razzled_electron_postcut.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Between Bucket Cut</h1>

df_hnl_prim = get_primary_shw_df(df_hnl)

df_nu_prim = get_primary_shw_df(df_nu)

df_cos_prim = get_primary_shw_df(df_cos)

In [ ]:
width = 19.00

df_hnl_prim['mod_t'] = df_hnl_prim['slc_opt0_time_corrected_Z_pandora']%width
df_nu_prim['mod_t'] = df_nu_prim['slc_opt0_time_corrected_Z_pandora']%width
df_cos_prim['mod_t'] = df_cos_prim['slc_opt0_time_corrected_Z_pandora']%width
#----------------------------------------------

hist, bins = plot_slc_var(
                    df_hnl_prim
                    , df_nu_prim
                    , df_cos_prim
                    , true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 19, xnbin = 19,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]'
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_precut.png", dpi = 200)
#-----------------------------------#
plt.show()

In [ ]:
ifTune = True
if ifTune:
    vary_Modt(df_hnl, df_nu, df_cos, true_counts, start_counts)
    if ifSave:
        plt.savefig(savePath+"/modt_tuning.png", dpi =200)

<h3>Apply Cut</h3>

In [ ]:
df_hnl = cutModt(df_hnl, 9)
df_nu = cutModt(df_nu, 9)
df_cos = cutModt(df_cos, 9)

<h3>Save Purity/Eff</h3>

In [ ]:
purity, eff = calc_purity_eff(df_hnl, df_nu, df_cos, true_counts, start_counts)

purity_arr.append(purity)
eff_arr.append(eff)
cut_arr.append("Between Bucket")

print(purity_arr)
print(eff_arr)
print(cut_arr)

In [ ]:
hist, bins = plot_slc_var(df_hnl, df_nu, df_cos,
                    true_counts, start_counts, 
                    'mod_t', 
                    scaleHNLPlot,
                    xmin = 0, xmax = 20, xnbin = 20,
                    xtitle = 'Opt0 Time Corrected Z % ' + str(width) + ' [ns]'
                    )

#-----------------------------------#
if ifSave:
    #fig.tight_layout()
    plt.savefig(savePath+"beam_bucket_final.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Plot Purity/Efficiency </h1>

In [ ]:
print(eff_arr)
print(purity_arr)
print(cut_arr)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(10, 6), sharex = True)
ax2 = ax1.twinx()

t = np.arange(0, len(cut_arr) , 1)

effcol = col_dict['MintGreen']
pcol = col_dict['Flamingo']

ax1.plot(t, eff_arr, c = effcol, marker = 'o', markersize = 5, ls='--')
ax2.plot(t, purity_arr, c = pcol, marker = 'o', markersize = 5, ls='--')

ax1.set_ylim(-5,105)
ax2.set_ylim(-5,105)

ax1.grid(color='gainsboro', which='both', axis='both', linestyle='--')

ax1.set_ylabel("Selection Effciency [%]", c = effcol, fontsize = 16)
ax2.set_ylabel("Purity [%]", c = pcol, fontsize = 16)

plt.xticks(range(0, len(t)))
ax1.set_xticklabels(cut_arr, rotation = 45, ha="right", fontsize = 16)

ax1.tick_params(bottom = True, top = True, left = True, right = False)
ax2.tick_params(bottom = True, top = True, left = False, right = True)

ax1.tick_params(axis = 'x', labelsize = 16, direction = 'in')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'in')
ax2.tick_params(axis = 'y', labelsize = 16, direction = 'in')

fig.tight_layout()

#-----------------------------------#
if ifSave:
    plt.savefig(savePath+"peff.png", dpi = 200)
#-----------------------------------#
plt.show()

<h1>Save Only Primary Showers </h1>

In [ ]:
df_hnl = get_primary_shw_df(df_hnl)

df_nu = get_primary_shw_df(df_nu)

df_cos = get_primary_shw_df(df_cos)
#df_cos_prim_shw2 = get_primary_shw_df(df_cos)

In [ ]:
df_hnl

In [ ]:
df_nu

In [ ]:
df_cos

In [ ]:
df_hnl[(df_hnl['run'] == 1) & (df_hnl['subrun']==790) & (df_hnl['event']==39)]

<h1> Save Cut Dataframe to Pickle</h1>

In [ ]:
df_hnl.to_pickle("./df_m200_v2_hnl.pkl", protocol = 5)

In [ ]:
df_nu.to_pickle("./df_m200_v2_nu.pkl", protocol = 5)

In [ ]:
df_cos.to_pickle("./df_m200_v2_cos.pkl", protocol = 5)